# 1º) CONEXION SQL PARA BBDD PARADAS DE BICIS

In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import requests
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

sqlalchemy.__version__

'1.4.32'

In [ ]:
#connection_string = 'mysql+pymysql://ironhack_user:%Vq=c>G5@173.201.189.217/BiciMAD'
#connection_string

In [ ]:
#engine = create_engine(connection_string)

In [ ]:
#inspector = inspect(engine)
#inspector.get_table_names()

In [ ]:
#df = pd.read_sql_query("SELECT * FROM bicimad_stations", engine)
#df

In [2]:
url_csv_bic= '../datasets/bicimad_stations.csv'

In [3]:
df = pd.read_csv(url_csv_bic, sep=',')

In [4]:
df.head()

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"


In [5]:
#df con la información que necesitamos

df_bic = df[['name', 'address', 'geometry.coordinates']]
df_bic

,name,address,geometry.coordinates
0,1a - Puerta del Sol A,Puerta del Sol nº 1,"[-3.7018341, 40.4172137]"
1,1b - Puerta del Sol B,Puerta del Sol nº 1,"[-3.701602938060457, 40.41731271011562]"
2,2 - Miguel Moya,Calle Miguel Moya nº 1,"[-3.7058415, 40.4205886]"
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,"[-3.7069171, 40.4302937]"
4,4 - Malasaña,Calle Manuela Malasaña nº 5,"[-3.7025875, 40.4285524]"
...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,"[-3.72997, 40.43896]"
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72699, 40.44375]"
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72693, 40.44342]"
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,"[-3.7272945, 40.4483322]"


In [6]:
df_bic[['location.longitude.bic', 'location.latitude.bic']]= df_bic['geometry.coordinates'].str.split(',', expand=True)

In [7]:
df_bic['location.longitude.bic'] = df_bic['location.longitude.bic'].apply(lambda x: float(x[1:]))
df_bic['location.latitude.bic'] = df_bic['location.latitude.bic'].apply(lambda x: float(x[:-1]))

### DF DEFINITIVO PARADAS DE BICIS = df_bic

In [8]:
df_bic = df_bic[['name', 'address', 'location.latitude.bic', 'location.longitude.bic']]
df_bic

,name,address,location.latitude.bic,location.longitude.bic
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,40.430294,-3.706917
4,4 - Malasaña,Calle Manuela Malasaña nº 5,40.428552,-3.702587
...,...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,40.448332,-3.727295


# 2º) CONEXIÓN API MONUMENTOS DE MADRID

In [9]:
endpoint = 'https://datos.madrid.es/egob'
url = '/catalogo/300356-0-monumentos-ciudad-madrid.json'
response = requests.get(endpoint + url)
#response
#print(type(response))

In [10]:
status = response.status_code
status

200

In [11]:
json_data = response.json()
print(type(json_data['@graph']))
#print(len(json_data))
#json_data.keys()

<class 'list'>


In [12]:
df = pd.DataFrame(pd.json_normalize(json_data['@graph']))
#df.head(5)

In [15]:
#ojo que hay nulos en localizaciones que será clave para obtener luego la más cercana.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   @id                             1848 non-null   object 
 1   id                              1848 non-null   object 
 2   title                           1848 non-null   object 
 3   relation                        1848 non-null   object 
 4   references                      1848 non-null   object 
 5   address.district.@id            1845 non-null   object 
 6   address.locality                1845 non-null   object 
 7   address.postal-code             1848 non-null   object 
 8   address.street-address          1848 non-null   object 
 9   location.latitude               1792 non-null   float64
 10  location.longitude              1792 non-null   float64
 11  organization.organization-desc  1848 non-null   object 
 12  organization.organization-name  18

In [16]:
#Revisando los valores nulos de la tabla

null_cols = df.isnull().sum()
null_cols[null_cols > 0]

address.district.@id      3
address.locality          3
location.latitude        56
location.longitude       56
address.area.@id        137
dtype: int64

### DF DEFINITIVO MONUMENTOS MADRID = df_monu

In [18]:
# vamos a crear un df con las columnas e información que nos interesan eliminando todos aquellos destinos que no tengan
# coordenadas.

df_monu = df[['title', 'address.street-address', 'location.latitude' , 'location.longitude']]
df_monu = df_monu.dropna()

In [19]:
df_monu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1792 entries, 0 to 1847
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   title                   1792 non-null   object 
 1   address.street-address  1792 non-null   object 
 2   location.latitude       1792 non-null   float64
 3   location.longitude      1792 non-null   float64
dtypes: float64(2), object(2)
memory usage: 70.0+ KB


# 3º) DISTANCIA MENOR ENTRE PUNTOS

### Funciones facilitadas

In [20]:
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas

In [21]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

### Test de las funciones

In [ ]:
to_mercator(df_bic['location.latitude.bic'][0], df_bic['location.longitude.bic'][0])

In [ ]:
to_mercator(df_monu['location.latitude'][0], df_monu['location.longitude'][0])

In [ ]:
distance_meters(df_monu['location.latitude'][0], df_monu['location.longitude'][0], df_bic['location.latitude.bic'][0], df_bic['location.longitude.bic'][0])

### Magia

In [22]:
%%time

df_nuevo = df_monu.head(1)
df_test = pd.concat([df_nuevo, df_bic], axis=1).ffill()
df_test['Distancia'] = df_test.apply(lambda x: distance_meters(x['location.latitude'], x['location.longitude'], x['location.latitude.bic'], x['location.longitude.bic']), axis=1)
zz = df_test.sort_values(by = 'Distancia')


CPU times: total: 5.05 s
Wall time: 5.09 s


In [23]:
zz.head(5)

,title,address.street-address,location.latitude,location.longitude,name,address,location.latitude.bic,location.longitude.bic,Distancia
232,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,230 - Carlota O’Neill,Calle Carlota O'Neill nº 32,40.443705,-3.648958,5559.887276
217,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,215 - Gutierre de Cetina,Calle Gutierre de Cetina nº 77,40.429706,-3.640994,5682.027353
231,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,229 - C.D. Concepción,Calle Jose del Hierro nº 5,40.437151,-3.648359,5841.282012
234,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,232 - José María Pereda,Calle José María de Pereda nº 1,40.432721,-3.648531,6126.918816
233,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,231 - Derechos Humanos,Calle Derechos Humanos nº 12,40.439657,-3.655766,6438.264535


In [ ]:
%%time

df_nuevo = df_monu.head(2)
df_test = pd.concat([df_nuevo, df_bic], axis=1).ffill()
df_test['Distancia'] = df_test.apply(lambda x: distance_meters(x['location.latitude'], x['location.longitude'], x['location.latitude.bic'], x['location.longitude.bic']), axis=1)
zz2 = df_test.sort_values(by = 'Distancia')

In [ ]:
zz2.head()

In [24]:
df_bic['key'] = 0
df_monu['key'] = 0
df_pre = df_bic.merge(df_monu, on='key', how='left')

In [25]:
df_pre = df_bic.merge(df_monu, on='key', how='left')#, lsuffix= '_1', rsuffix= '_2')

In [ ]:
%%time

#df_pre['Distancia'] = df_pre.apply(lambda x: distance_meters(x['location.latitude'], x['location.longitude'], 
                                                            # x['location.latitude.bic'], x['location.longitude.bic']), axis=1)

In [ ]:
data = df_pre.to_csv('../datasets/df.csv', index=False)

### Ya tenemos el DF_pre guardado

In [26]:
url_df_pre = '../datasets/df.csv'

In [27]:
df_def = pd.read_csv(url_df_pre, sep=',')

In [28]:
df_def.head()

,name,address,location.latitude.bic,location.longitude.bic,key,title,address.street-address,location.latitude,location.longitude,Distancia
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,A las víctimas del Holocausto,JARDIN TRES CULTURAS,40.464322,-3.603558,12152.077352
1,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,A los Abuelos,C Alicún,40.479812,-3.661719,8281.468285
2,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,A Neruda-El Ser alado,Gta José Luis Ozores 116,40.381855,-3.652419,6773.299650
3,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,Abogados Laboristas,C Atocha 55,40.412911,-3.700017,520.142426
4,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834,0,Abstracta I,,40.421746,-3.662293,4439.554041


In [29]:
zz5 = df_def.sort_values(by = ['title', 'Distancia']).rename(columns={'title':'Place of Interest', 
                                                                      'address.street-address':'Place address',
                                                                     'name':'BiciMAD station',
                                                                      'address':'Station Location'})


zz5['Type of place'] = np.nan
zz5['Type of place'] = zz5['Type of place'].fillna('Monumento de la ciudad de Madrid')
df_final = zz5[['Place of Interest', 'Type of place', 'Place address', 'BiciMAD station', 'Station Location']] #.info()
df_final

,Place of Interest,Type of place,Place address,BiciMAD station,Station Location
410370,A Neruda-El Ser alado,Monumento de la ciudad de Madrid,Gta José Luis Ozores 116,227 - Concordia,Calle Arroyo del Olivar nº 6
408578,A Neruda-El Ser alado,Monumento de la ciudad de Madrid,Gta José Luis Ozores 116,226 - Sierra Toledana,Camino de Valderribas nº 94
406786,A Neruda-El Ser alado,Monumento de la ciudad de Madrid,Gta José Luis Ozores 116,225 - Doctor Lozano,Calle Sierra Toledana nº 33
324354,A Neruda-El Ser alado,Monumento de la ciudad de Madrid,Gta José Luis Ozores 116,179 - Puente de Vallecas,Avenida de la Albufera nº 6
397826,A Neruda-El Ser alado,Monumento de la ciudad de Madrid,Gta José Luis Ozores 116,220 - Plaza del Encuentro,Calle Arroyo de las Pilillas nº 6
...,...,...,...,...,...
392447,Órgano,Monumento de la ciudad de Madrid,C Castelló 77,216 - Puerta del Ángel,Plaza Puerta del Ángel nº 4
394239,Órgano,Monumento de la ciudad de Madrid,C Castelló 77,217 - Pedro Rico,Calle Pedro Rico nº 4
473087,Órgano,Monumento de la ciudad de Madrid,C Castelló 77,261 - Facultad Derecho,Avenida Complutense nº 23
406783,Órgano,Monumento de la ciudad de Madrid,C Castelló 77,224 - Caramuel,Calle Caramuel nº 51


### Dataframe Final

In [30]:
df_final['BiciMAD station'] = df_final['BiciMAD station'].apply(lambda x: x.split('-')[1])
df_final

,Place of Interest,Type of place,Place address,BiciMAD station,Station Location
410370,A Neruda-El Ser alado,Monumento de la ciudad de Madrid,Gta José Luis Ozores 116,Concordia,Calle Arroyo del Olivar nº 6
408578,A Neruda-El Ser alado,Monumento de la ciudad de Madrid,Gta José Luis Ozores 116,Sierra Toledana,Camino de Valderribas nº 94
406786,A Neruda-El Ser alado,Monumento de la ciudad de Madrid,Gta José Luis Ozores 116,Doctor Lozano,Calle Sierra Toledana nº 33
324354,A Neruda-El Ser alado,Monumento de la ciudad de Madrid,Gta José Luis Ozores 116,Puente de Vallecas,Avenida de la Albufera nº 6
397826,A Neruda-El Ser alado,Monumento de la ciudad de Madrid,Gta José Luis Ozores 116,Plaza del Encuentro,Calle Arroyo de las Pilillas nº 6
...,...,...,...,...,...
392447,Órgano,Monumento de la ciudad de Madrid,C Castelló 77,Puerta del Ángel,Plaza Puerta del Ángel nº 4
394239,Órgano,Monumento de la ciudad de Madrid,C Castelló 77,Pedro Rico,Calle Pedro Rico nº 4
473087,Órgano,Monumento de la ciudad de Madrid,C Castelló 77,Facultad Derecho,Avenida Complutense nº 23
406783,Órgano,Monumento de la ciudad de Madrid,C Castelló 77,Caramuel,Calle Caramuel nº 51


In [31]:
df_final['Place of Interest'].unique()

array(['A Neruda-El Ser alado', 'A las víctimas del Holocausto',
       'A los Abuelos', ..., 'Árbol de la Vida', 'Íñigo Arista', 'Órgano'],
      dtype=object)

### Opción 1 donde el usuario elige un punto de interés en concreto

In [32]:
def select_point_of_interest(x, y): # en X entrará el nombre del punto de interés. La Y en número de estaciones cercanas.
    is_poi = df_final.loc[:, 'Place of Interest'] == x # zz5 cambialo por el nombre de tu dataframe. El resto se puede quedar igual
    df_poi = df_final.loc[is_poi]
    df_poi = df_poi.reset_index(drop=True)
    return df_poi.head(y) # si solo quieres el punto más cercano pon un 1 y sino 3 para los 3 más cercanos

In [33]:
select_point_of_interest('A las víctimas del Holocausto', 2)

,Place of Interest,Type of place,Place address,BiciMAD station,Station Location
0,A las víctimas del Holocausto,Monumento de la ciudad de Madrid,JARDIN TRES CULTURAS,Carlota O’Neill,Calle Carlota O'Neill nº 32
1,A las víctimas del Holocausto,Monumento de la ciudad de Madrid,JARDIN TRES CULTURAS,Gutierre de Cetina,Calle Gutierre de Cetina nº 77


### Opción 2 donde se muestran las estaciones más cercanas a cada punto de interés

In [34]:
test = df_final.loc[:, 'Place of Interest'] == 'A Neruda-El Ser alado'
test1 = df_final.loc[test]
test1 = test1.reset_index(drop=True)
test1 = test1.head(0)
test1

,Place of Interest,Type of place,Place address,BiciMAD station,Station Location


In [ ]:
test_2 = df_final.loc[:, 'Place of Interest'] == 'A las víctimas del Holocausto'
test2 = df_final.loc[test_2]
test2 = test2.reset_index(drop=True)
test2 = test2.head(1)
test2

In [ ]:
ff = test1.append(test2, ignore_index=True)
ff

In [ ]:
test_3 = df_final.loc[:, 'Place of Interest'] == 'A los Abuelos'
test3 = df_final.loc[test_3]
test3 = test3.reset_index(drop=True)
test3 = test3.head(1)
test3

In [ ]:
ff = ff.append(test3, ignore_index=True)
ff

In [ ]:
poi_uniques = df_final['Place of Interest'].unique().tolist()
#poi_uniques

In [ ]:
%%time

for i in poi_uniques:
    test_2 = df_final.loc[:, 'Place of Interest'] == i
    test2 = df_final.loc[test_2]
    test2 = test2.reset_index(drop=True)
    test2 = test2.head(3)
    test1 = test1.append(test2, ignore_index=True)
test1.head()

# 4º RESUMEN FINAL

## PIPELINE SECUENCIAL

### Conexción y manipulación de Data Frame final de bicis

In [ ]:
#ip #crear una función que le indique desde qué ip conectarse (clase=173.201.189.217 , casa=92.172.38.71)
#connection_string = 'mysql+pymysql://ironhack_user:%Vq=c>G5@173.201.189.217/BiciMAD'
#engine = create_engine(connection_string)
#inspector = inspect(engine)
#dfb = pd.read_sql_query("SELECT * FROM bicimad_stations", engine)
url_csv_bic= '../datasets/bicimad_stations.csv' # esto se tendría que desconectar si pudiéramos conectarnos en clase
dfb = pd.read_csv(url_csv_bic, sep=',') # esto se tendría que desconectar si pudiéramos conectarnos en clase
df_bic = dfb[['name', 'address', 'geometry.coordinates']]
df_bic[['location.longitude.bic', 'location.latitude.bic']]= df_bic['geometry.coordinates'].str.split(',', expand=True)
df_bic['location.longitude.bic'] = df_bic['location.longitude.bic'].apply(lambda x: float(x[1:]))
df_bic['location.latitude.bic'] = df_bic['location.latitude.bic'].apply(lambda x: float(x[:-1]))
df_bic = df_bic[['name', 'address', 'location.latitude.bic', 'location.longitude.bic']]

### Conexión y manipulación de Data Frame Monumentos de Madrid

In [ ]:
endpoint = 'https://datos.madrid.es/egob'
url = '/catalogo/300356-0-monumentos-ciudad-madrid.json'
response = requests.get(endpoint + url)
json_data = response.json()
dfm = pd.DataFrame(pd.json_normalize(json_data['@graph']))
df_monu = dfm[['title', 'address.street-address', 'location.latitude' , 'location.longitude']]
df_monu = df_monu.dropna()

### Merge entre ambos DataFrames

In [ ]:
df_bic['key'] = 0
df_monu['key'] = 0
df_pre = df_bic.merge(df_monu, on='key', how='left')

### Aplicación de mercator a todos los valores posibles y creación de CSV Final para realizar las consultas

In [ ]:
#df_pre['Distancia'] = df_pre.apply(lambda x: distance_meters(x['location.latitude'], x['location.longitude'], 
                                                            # x['location.latitude.bic'], x['location.longitude.bic']), axis=1)
#data = df_pre.to_csv('../datasets/df.csv', index=False)

### Manipulación de Dataframe Final para la obtención de los resultados requeridos

In [ ]:
url_df_pre = '../datasets/df.csv'
df_def = pd.read_csv(url_df_pre, sep=',')
zz5 = df_def.sort_values(by = ['title', 'Distancia']).rename(columns={'title':'Place of Interest', 
                                                                      'address.street-address':'Place address',
                                                                     'name':'BiciMAD station',
                                                                      'address':'Station Location'})
zz5['Type of place'] = np.nan
zz5['Type of place'] = zz5['Type of place'].fillna('Monumento de la ciudad de Madrid')
df_final = zz5[['Place of Interest', 'Type of place', 'Place address', 'BiciMAD station', 'Station Location']]
df_final['BiciMAD station'] = df_final['BiciMAD station'].apply(lambda x: x.split('-')[1])

### Opción 1 donde el usuario indica un lugar de interés específico

In [ ]:
is_poi = df_final.loc[:, 'Place of Interest'] == 'A los Abuelos'
df_poi = df_final.loc[is_poi]
df_poi = df_poi.reset_index(drop=True)
df_poi.head(1)
csv_poi = df_poi.head(1).to_csv('../datasets/specific_point_of_interest.csv', index=False)

### Opción 2 donde el usuario indica listado de todos los puntos de interés con su estación más cercana

In [ ]:
poi_uniques = df_final['Place of Interest'].unique().tolist()
test = df_final.loc[:, 'Place of Interest'] == 'A Neruda-El Ser alado'
test1 = df_final.loc[test]
test1 = test1.reset_index(drop=True)
test1 = test1.head(0)
for i in poi_uniques:
    test_2 = df_final.loc[:, 'Place of Interest'] == i
    test2 = df_final.loc[test_2]
    test2 = test2.reset_index(drop=True)
    test2 = test2.head(1)
    test1 = test1.append(test2, ignore_index=True)
test1.head()
csv_full = test1.to_csv('../datasets/all_points_of_interests.csv', index=False)

## PIPELINE FUNCIONAL

### Acquisition

In [ ]:
#las dos opciones serían 'mysql' o 'csv' y se podría cambiar el path, query e ip (casa = '92.172.38.71')
def connec_mysql(query="SELECT * FROM bicimad_stations", ip='173.201.189.217', db='/BiciMAD', 
                 root='mysql+pymysql://ironhack_user:', pss='%Vq=c>G5@'): 
    connection_string = root+pss+ip+db
    engine = create_engine(connection_string)
    inspector = inspect(engine)
    df = pd.read_sql_query(query, engine)
    return df

In [2]:
def connec_csv(path):
    # función para únicamente tomar en formato csv la data sin tratar de una conexión a bbdd mysql
    # para nuestro proyecto serían path='../datasets/bicimad_stations.csv' o '../datasets/df.csv' (renombrar el archivo a df_merged)
    df = pd.read_csv(path, sep=',')
    return df

In [ ]:
def connec_api_mad(endpoint = 'https://datos.madrid.es/egob', url = '/catalogo/300356-0-monumentos-ciudad-madrid.json'):    
    response = requests.get(endpoint + url)
    json_data = response.json()
    df = pd.DataFrame(pd.json_normalize(json_data['@graph']))
    return df

### Manipulación de DF

In [3]:
def selec_cols(df, cols=['name', 'address', 'geometry.coordinates']):
    # realiza un slicing sobre un df en base a las colummnas
    # para nuestro proyecto cols=['name', 'address', 'geometry.coordinates'] o 
    # cols=['title', 'address.street-address', 'location.latitude' , 'location.longitude']
    # cols=['Place of Interest', 'Type of place', 'Place address', 'BiciMAD station', 'Station Location']
    df = df[cols]
    return df

In [4]:
def dividir_col(df, create_cols=['location.longitude.bic', 'location.latitude.bic'], div_col='geometry.coordinates', s=','):
    # función que reparte en dos columnas la data una columna que contiene listas de dos elementos convirtiéndolos en strings
    df[create_cols]= df[div_col].str.split(s, expand=True)
    return df 

In [5]:
def float_slicing(df, col, pos1=None, pos2=None):
    # convierte string en float pudiendo hacer un slicing del elemento en esa posición.
    # para nuestro proyecto tendríamos col='location.longitude.bic' o col='location.latitude.bic' y pos1=1 o pos2=-1
    df[col] = df[col].apply(lambda x: float(x[pos1:pos2]))
    return df

In [ ]:
def drop_na(df):
    # función para eliminar todas las filas que contengan valores NaN
    df = df.dropna()
    return df

In [ ]:
def sort_df (df, head=['title', 'Distancia'], order):
    # función para ordenar un df en base a columnas y de manera ascendente o descendente con True o False en order.
    df = df.sort_values(by = head, ascending=order)
    return df

In [ ]:
def rename_col (df, keys, values):
    # función que renombra tanta columnas como queramos siempre que el número de keys y values coincida
    # para nuestro proyecto sería dict_from_list = {'title':'Place of Interest', 'address.street-address':'Place address',
                                                    #'name':'BiciMAD station','address':'Station Location'}
    key_list = list(keys)
    value_list = list(values)
    dict_from_list = dict(zip(key_list, value_list))
    df = df.rename(columns=dict_from_list)
    return df

In [ ]:
def newcol_values(df, new_col='Type of place', fill='Monumento de la ciudad de Madrid'):
    # función que crea una nueva columna a la que se le asigna el mismo valor a todas las filas.
    df[new_col] = np.nan
    df[new_col] = df[new_col].fillna(fill)
    return df

In [ ]:
def split(df, col='BiciMAD station', sp='-', pos=1):
    # función para realizar limpieza en un string de los valores de una columna, haciendo un split y slicing a guardar
    df[col] = df[col].apply(lambda x: x.split(sp)[pos])
    return df

In [ ]:
def merge_df(df_bic_an, df_monu_an, col='key'):
    # función que crea una columan key con valores 0 en dos dataframes para poder realizar un merge
    df_bic_an[col] = 0
    df_monu_an[col] = 0
    df = df_bic_an.merge(df_monu_an, on=col, how='left')
    return df

### Operaciones Aritméticas

In [ ]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [ ]:
def df_dist(df_pre_mercator, col='Distancia', lata='location.latitude', longa='location.longitude', 
            latb='location.latitude.bic', longb='location.longitude.bic'):
    # función que crea una columna nueva cuyos valores son los resultados de aplicar mercator con la data de 4 columnas.
    df_pre_mercator[col] = df_pre_mercator.apply(lambda x: distance_meters(x[lata], x[longa], x[latb], x[longb]), axis=1)   
    return df_pre_mercator

### Export total distancias

In [ ]:
def save_df_csv(df_pre_mer, path='../datasets/df_merged.csv'): #ver si crear una carpeta donde guardar esta data
    # función que convierte un df en csv
    data = df_pre_mer.to_csv(path, index=False)
    return data

### Reporting

In [ ]:
def point_interest(df_fin, ask, path='../datasets/specific_point_of_interest.csv', theme='Place of Interest', num):
    # Función que nos devuelve n resultados en base al theme del usuario.
    # analizar cómo poder elegir el theme haciendo que el sort anterior sea posterior al rename de columnas.
    is_poi = df_fin.loc[:, theme] == ask
    df_poi = df_fin.loc[is_poi]
    df_poi = df_poi.reset_index(drop=True)
    csv_poi = df_poi.head(num).to_csv(path, index=False)
    return df_poi.head(num)

In [ ]:
def all_monuments(df_fin, bics=1, path='../datasets/all_points_of_interests.csv', theme='Place of Interest'):
    # función que nos devuelve un listado de n resultados por cada theme indicado por el usuario
    poi_uniques = df_fin[theme].unique().tolist()
    df = df_fin.loc[:, 'Place of Interest'] == 'A Neruda-El Ser alado'
    df1 = df_fin.loc[df]
    df1 = df1.reset_index(drop=True)
    df1 = df1.head(0)
    for i in poi_uniques:
        df_2 = df_fin.loc[:, theme] == i
        df2 = df_fin.loc[df_2]
        df2 = df2.reset_index(drop=True)
        df2 = df2.head(bics)
        df1 = df1.append(df2, ignore_index=True)
    csv_full = test1.to_csv(path, index=False)
    return df1.head()